In [20]:
import torch
import clip
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import CLIPProcessor, CLIPModel
from datasets import load_dataset
from PIL import Image

In [3]:
all_embeddings = np.load('combined_embeddings.npy')

In [4]:
all_embeddings = all_embeddings / np.linalg.norm(all_embeddings, axis=1, keepdims=True)

In [10]:
ds = load_dataset("Artificio/WikiArt")

In [11]:
train_data = ds['train']

In [28]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image

# Load model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load the image using PIL
image_path = "img.png"  # Replace with your actual image path
image = Image.open(image_path)  # Open the image file

# Example text
text = "find similar line strokes"

# Preprocess inputs
inputs = processor(text=[text], images=image, return_tensors="pt", padding=True)

# Move the inputs to the device (GPU/CPU)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Get the image and text embeddings
with torch.no_grad():
    # Extract the image features and text features correctly
    image_features = model.get_image_features(pixel_values=inputs['pixel_values'])
    text_features = model.get_text_features(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

# Normalize the embeddings
image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

# Now you can compute similarity between the text and image embeddings


In [32]:
weight_img = 0.65
weight_text = 0.35

In [34]:
image_features = image_features.cpu().detach().numpy()
text_features = text_features.cpu().detach().numpy()

In [38]:
final_embedding = weight_img * image_features + weight_text * text_features

In [39]:
similarities = cosine_similarity(final_embedding, all_embeddings)

TypeError: 'numpy.ndarray' object is not callable